### Steps
1) load document 
2) split
3) Make vector Store
4) Make retriever
5) Choose local LLM
6) Prompt Engineering
7) Create RAG Chain

# 1) OllamaLLm and HuggingFaceEmbedding: mistralai/Mistral-7B-Instruct-v04

## Document Loader
    pdf loader : langchain inbuilt document loader 

In [22]:
from langchain_community.document_loaders import PyPDFLoader
file_path = ("Leave_Policy_2024.pdf")
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()
len(pages)


4

## Split
    smaller chunks

In [23]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)
len(splits)


15

In [24]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

## Create vector store
    stores embeddings of Documents

In [28]:
from langchain_huggingface import HuggingFaceEmbeddings

In [29]:
## Locally loading embedding
### Still pending --------------------------->>>>>>>>>>>>>>
# new_embeddings = HuggingFaceEmbeddings(cache_folder= r"C:\Users\30078206\Downloads\sentence-transformersall-mpnet-base-v2")


In [30]:
## Not local
new_embeddings = HuggingFaceEmbeddings(model_name= "sentence-transformers/all-mpnet-base-v2")
vectorstore = Chroma.from_documents(documents=splits, embedding=new_embeddings)
vectorstore

c:\Users\30078206\AppData\Local\anaconda3\envs\myenv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Create retriever

In [31]:
retriever = vectorstore.as_retriever(search_type="similarity")

In [32]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

### LLM Used:
local google flan t5
 - Huggingface model, loaded locally
 


In [14]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,pipeline
from langchain_huggingface import HuggingFacePipeline
model_id = "google/flan-t5-small"
model_path = r"D:\OneDrive - Adani\Desktop\LEARNING_FOLDER\_Kolkata_2024\1_LLM\local_downloaded_models\flan_small"


tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path= model_path)

model = AutoModelForSeq2SeqLM.from_pretrained(model_path)


pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens = 512)
llm = HuggingFacePipeline(pipeline=pipe)

### Creating Prompt

In [16]:
from langchain.prompts import PromptTemplate

In [17]:

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)


### New Terms:
- Multiqueryretriever: 
    - automates process of tuning
    - to generate multiple queries from different perspective 
    - for each query- returns relevant documents,, takes union across all 
    - Overcomes the limitation of distance based retrieval


In [19]:
from langchain.retrievers.multi_query import MultiQueryRetriever


In [33]:
retriever = MultiQueryRetriever.from_llm(
    retriever, 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [34]:
import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

In [35]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [40]:
questions = chain.invoke("Summarize the document")

INFO:langchain.retrievers.multi_query:Generated queries: ['What is the document that is a vector database?']


In [41]:
questions

'PL is the entitlement of PL to 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days in a Leave Year. For nexample, the entitlement of PL is 21 days 

In [36]:
import re

In [ ]:
answer =re.sub(r'[0-9]','',questions)
question_list = answer.split('\n')
question_list

In [ ]:
chain.invoke(question_list[0])

In [ ]:
''' Taking too much time'''
# answer_list = []
# for question in question_list:
#     ans = chain.invoke(question)
#     answer_list.append(ans)

In [ ]:
print("ANSWERS-----")
# print(answer_list)

In [ ]:
question_bank = [
    "Give me a summary of the document",
    "Tell me all about the leaves",
    "Make a list of questions from the document"
]

--------------------------------------------(2)--------------------------------------------